In [1]:
import numpy as np, pandas as pd
from matplotlib.pyplot import subplots
from sklearn.linear_model import (LinearRegression, LogisticRegression, Lasso)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import (train_test_split, GridSearchCV)
from ISLP import load_data
from ISLP.torch import SimpleDataModule, SimpleModule, ErrorTracker

import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset

# torchmetrics can compare various metrics to evaluate model
from torchmetrics import MeanAbsoluteError, R2Score
# torchinfo provides useful summary of layers of a model
from torchinfo import summary
# pytorch_lightning simplifies model fitting by reducing boilerplate
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger
from torchvision.transforms import Resize, Normalize, CenterCrop, ToTensor

In [2]:
# Time Series Prediction on Stock Market Data
NYSE = load_data('NYSE')

# Adding month factor
int(NYSE.index[0].split('-')[1])
month = []
for date in NYSE.index:
    month.append(int(date.split('-')[1]))
NYSE['month'] = month

cols = ['DJ_return', 'log_volume','log_volatility','month']
X = pd.DataFrame(StandardScaler(with_mean=True, with_std=True)
                .fit_transform(NYSE[cols]),
                 columns=cols,
                 index=NYSE.index)

In [3]:
# Setup lagged versions of data
for lag in range(1,6):
    for col in cols:
        newcol = np.zeros(X.shape[0]) * np.nan
        newcol[lag:] = X[col].values[:-lag] 
        X.insert(len(X.columns), "{0}_{1}".format(col, lag), newcol)
X.insert(len(X.columns), 'train', NYSE['train'])
X = X.dropna()

In [4]:
# Extract response, training indicator 
Y, train = X['log_volume'], X['train']
X = X.drop(columns=['train'] + cols) # Drop today's data 
X.columns

Index(['DJ_return_1', 'log_volume_1', 'log_volatility_1', 'month_1',
       'DJ_return_2', 'log_volume_2', 'log_volatility_2', 'month_2',
       'DJ_return_3', 'log_volume_3', 'log_volatility_3', 'month_3',
       'DJ_return_4', 'log_volume_4', 'log_volatility_4', 'month_4',
       'DJ_return_5', 'log_volume_5', 'log_volatility_5', 'month_5'],
      dtype='object')

In [5]:
# Reshape data for RNN
ordered_cols = []
for lag in range(5, 0 , -1):
    for col in cols:
        ordered_cols.append('{0}_{1}'.format(col, lag))
X = X.reindex(columns=ordered_cols)
X.columns

Index(['DJ_return_5', 'log_volume_5', 'log_volatility_5', 'month_5',
       'DJ_return_4', 'log_volume_4', 'log_volatility_4', 'month_4',
       'DJ_return_3', 'log_volume_3', 'log_volatility_3', 'month_3',
       'DJ_return_2', 'log_volume_2', 'log_volatility_2', 'month_2',
       'DJ_return_1', 'log_volume_1', 'log_volatility_1', 'month_1'],
      dtype='object')

In [6]:
# Reshape data
X_rnn = X.to_numpy().reshape((-1,5,4)) # -1 means numpy.reshape() deduces size based on remaining args
X_rnn.shape

(6046, 5, 4)

In [7]:
# RNN with 12 hidden units and 10% dropout
class NYSEModel(nn.Module):
    def __init__(self):
        super(NYSEModel, self).__init__()
        self.rnn = nn.RNN(4, 15, batch_first=True)
        self.dense = nn.Linear(15, 1)
        self.dropout = nn.Dropout(0.4)
    def forward(self, x):
        val, h_n = self.rnn(x)
        val = self.dense(self.dropout(val[:,-1])) # -1 = last time point (today)
        return torch.flatten(val)
nyse_model = NYSEModel()

In [8]:
# Format data
datasets = []
for mask in [train, ~train]:
    X_rnn_t = torch.tensor(X_rnn[mask].astype(np.float32))
    Y_t = torch.tensor(Y[mask].astype(np.float32))
    datasets.append(TensorDataset(X_rnn_t, Y_t))
nyse_train, nyse_test = datasets

In [9]:
summary(nyse_model,
        input_data=X_rnn_t,
        col_names=['input_size',
                   'output_size',
                   'num_params'])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
NYSEModel                                [1770, 5, 4]              [1770]                    --
├─RNN: 1-1                               [1770, 5, 4]              [1770, 5, 15]             315
├─Dropout: 1-2                           [1770, 15]                [1770, 15]                --
├─Linear: 1-3                            [1770, 15]                [1770, 1]                 16
Total params: 331
Trainable params: 331
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 2.82
Input size (MB): 0.14
Forward/backward pass size (MB): 1.08
Params size (MB): 0.00
Estimated Total Size (MB): 1.22

In [10]:
# Put the two datasets into datamodule with batch size 64
nyse_dm = SimpleDataModule(nyse_train, nyse_test,
                           num_workers=5,
                           validation=nyse_test,
                           batch_size=64)

In [11]:
# Training
nyse_optimizer = RMSprop(nyse_model.parameters(),
                         lr=0.0005)
nyse_module = SimpleModule.regression(nyse_model,
                                      optimizer=nyse_optimizer,
                                      metrics={'r2':R2Score()}) # Requesting R^2 computed at each epochv

In [79]:
nyse_trainer = Trainer(deterministic=True,
                       max_epochs=200,
                       callbacks=[ErrorTracker()],
                       accelerator='mps')
nyse_trainer.fit(nyse_module,
                 datamodule=nyse_dm)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type      | Params | Mode 
--------------------------------------------
0 | model | NYSEModel | 331    | train
1 | loss  | MSELoss   | 0      | train
--------------------------------------------
331       Trainable params
0         Non-trainable params
331       Total params
0.001     Total estimated model params size (MB)


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

...

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.


In [80]:
nyse_trainer.test(nyse_module,
                  datamodule=nyse_dm)

Testing: |                                                | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.6168254613876343     │
│          test_r2          │    0.4146025776863098     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.6168254613876343, 'test_r2': 0.4146025776863098}]

In [12]:
# Test R^2 with one from lab dropout 0.1, hidden layers 12: 0.4151615500450134
# Test R^2 with month variable dropout 0.1, hidden layers 12 : 0.41357946395874023
# Test R^2 with month, dropout 0.2, hidden layers 15 : 0.42154204845428467
# Test R^2 with month, dropout 0.4, hidden layers 15 : 0.4146025776863098|

In [36]:
# Exercise 10: Linear AR Model
# Build NonLinearARModel()
class LinearARModel(nn.Module):
    def __init__(self):
        super(LinearARModel, self).__init__()
        self._forward = nn.Sequential(nn.Flatten(),
                                      nn.ReLU(),
                                      nn.Dropout(0.2),
                                      nn.Linear(20, 1))
    def forward(self, x):
        return torch.flatten(self._forward(x))

In [37]:
lin_model = LinearARModel()
lin_optimizer = RMSprop(lin_model.parameters(),
                           lr=0.001)
lin_module = SimpleModule.regression(lin_model,
                                        optimizer=lin_optimizer,
                                        metrics={'r2':R2Score()})

In [38]:
lin_trainer = Trainer(deterministic=True,
                     max_epochs=20,
                     callbacks=[ErrorTracker()],
                    accelerator='mps')
lin_trainer.fit(lin_module, datamodule=nyse_dm)
lin_trainer.test(lin_module, datamodule=nyse_dm)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type          | Params | Mode 
------------------------------------------------
0 | model | LinearARModel | 21     | train
1 | loss  | MSELoss       | 0      | train
------------------------------------------------
21        Trainable params
0         Non-trainable params
21        Total params
0.000     Total estimated model params size (MB)


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

...

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.7059076428413391     │
│          test_r2          │    0.3300591707229614     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.7059076428413391, 'test_r2': 0.3300591707229614}]

In [26]:
# Test R^2 is considerably worse than before : 0.330

4